In [1]:
pwd()

"/Users/nicole/code/FermiCG-data/he_clusters/he08_02"

In [2]:
using Pkg; Pkg.activate("../../../FermiCG/")
using FermiCG
using PyCall
using Plots
using LinearAlgebra
using Printf

  Activating environment at `~/code/FermiCG/Project.toml`


In [5]:
molecule = "
He         0.83846       -0.83846        0.83846
He         0.83846       -0.83846       -0.83846
He         0.83846        0.83846       -0.83846
He         0.83846        0.83846        0.83846
He        -0.83846        0.83846        0.83846
He        -0.83846        0.83846       -0.83846
He        -0.83846       -0.83846       -0.83846
He        -0.83846       -0.83846        0.83846
"
atoms = []
for (li,line) in enumerate(split(rstrip(lstrip(molecule)), "\n"))
    l = split(line)
    push!(atoms, Atom(li, l[1], parse.(Float64,l[2:4])))
end

#basis = "aug-cc-pvdz" #9 orbs on each He
basis = "cc-pvdz" #5 orbs on each He
#basis = "cc-pvtz" # 6 orbs on each He
#basis = "sto-3g"

# Create FermiCG.Molecule type
mol = Molecule(0,1,atoms,basis)

pyscf = pyimport("pyscf")
pymol = pyscf.gto.Mole(atom=molecule, spin=0, charge=0, basis=basis)
pymol.build()
mf = pyscf.scf.RHF(pymol).run()
s = mf.get_ovlp(pymol)

lo = pyimport("pyscf.lo.orth")
lo_ao = lo.lowdin(s)
println("size of Lowdin ortho AO's:", size(lo_ao))

#FermiCG.pyscf_write_molden(mol, lo_ao, filename="he08_02/lowdin_ao_augccpvdz.molden")

#write fci dump file from the modified mo coefficients
tools = pyimport("pyscf.tools")
tools.fcidump.from_mo(pymol, "fcidump.he08_02", lo_ao)

converged SCF energy = -22.7370439057951
size of Lowdin ortho AO's:(40, 40)


In [6]:
#Can just read in pyscf dump file for integrals (once you have already run an scf calculation)
pyscf = pyimport("pyscf");
fcidump = pyimport("pyscf.tools.fcidump");
ctx = fcidump.read("fcidump.he08_02");
h = ctx["H1"];
g = ctx["H2"];
ecore = ctx["ECORE"];
g = pyscf.ao2mo.restore("1", g, size(h,2))
#This one below was not working. Error: setfield! immutable struct of type InCoreInts cannot be changed
ints = InCoreInts(ecore,h,g);

#occ = [1, 10, 19, 28, 37, 46, 55, 64]
#Cocc = lo_ao[:,occ];
#ints = FermiCG.pyscf_build_ints(mol, lo_ao, 2.0*Cocc*Cocc');


In [13]:
#Define clusters and intial Fock space for inital CMF calc for 9 orbs each He
clusters_in = [(1:9),(10:18), (19:27), (28:36), (37:45), (46:54), (55:63), (64:72)]
init_fspace = [(1,1),(1,1),(1,1),(1,1),(1,1),(1,1),(1,1),(1,1)]
rdm1 = zeros(size(ints.h1))
na=8
nb=8

8

In [7]:
#Define clusters and intial Fock space for inital CMF calc for 5 orbs each He
clusters_in = [(1:5),(6:10), (11:15), (16:20), (21:25), (26:30), (31:35), (36:40)]
init_fspace = [(1,1),(1,1),(1,1),(1,1),(1,1),(1,1),(1,1),(1,1)]
rdm1 = zeros(size(ints.h1))
na=8
nb=8

8

In [24]:
#Define clusters and intial Fock space for inital CMF calc for 1 orb each He
clusters_in = [1, 2, 3, 4, 5, 6, 7, 8]
init_fspace = [(1,1),(1,1),(1,1),(1,1),(1,1),(1,1),(1,1),(1,1)]
rdm1 = zeros(size(ints.h1))
na=8
nb=8

8

In [8]:
#Define clusters now using FermiCG code
clusters = [Cluster(i,collect(clusters_in[i])) for i = 1:length(clusters_in)]
#clusters = [Cluster(i, [clusters_in[i]]) for i = 1:length(clusters_in)]
display(clusters)

print(size(ints.h1))
rdm1 = zeros(size(ints.h1))

#do a CMF calculation to optimize cluster orbitals

e_cmf, U, Da, Db = FermiCG.cmf_oo(ints, clusters, init_fspace, rdm1, rdm1, max_iter_oo=100, verbose=0, gconv=1e-6, method="bfgs");


8-element Vector{Cluster}:
 Cluster(1, [1, 2, 3, 4, 5])
 Cluster(2, [6, 7, 8, 9, 10])
 Cluster(3, [11, 12, 13, 14, 15])
 Cluster(4, [16, 17, 18, 19, 20])
 Cluster(5, [21, 22, 23, 24, 25])
 Cluster(6, [26, 27, 28, 29, 30])
 Cluster(7, [31, 32, 33, 34, 35])
 Cluster(8, [36, 37, 38, 39, 40])

(40, 40) ooCMF Iter:    1 Total= -22.753131993680 Active= -51.525945206536 G=     2.20e+00
 ooCMF Iter:    2 Total= -22.865940859299 Active= -51.638754072155 G=     1.71e+00
 ooCMF Iter:    3 Total= -22.905078783769 Active= -51.677891996625 G=     1.26e+00
 ooCMF Iter:    4 Total= -22.975068755215 Active= -51.747881968071 G=     5.02e-01
 ooCMF Iter:    5 Total= -22.985420419325 Active= -51.758233632182 G=     6.95e-02
 ooCMF Iter:    6 Total= -22.985658113294 Active= -51.758471326150 G=     2.52e-02
 ooCMF Iter:    7 Total= -22.985700569874 Active= -51.758513782730 G=     3.22e-02
 ooCMF Iter:    8 Total= -22.986193770313 Active= -51.759006983169 G=     6.67e-02
 ooCMF Iter:    9 Total= -22.986361735092 Active= -51.759174947948 G=     6.58e-02
 ooCMF Iter:   10 Total= -22.986486567579 Active= -51.759299780435 G=     5.56e-02
 ooCMF Iter:   11 Total= -22.986691476677 Active= -51.759504689533 G=     1.71e-02
 ooCMF Iter:   12 Total= -22.986699237884 Active= -51.759512450740 G=     1.42e

 * Status: success

 * Candidate solution
    Final objective value:     -2.298671e+01

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 5.01e-07 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.26e-05 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.23e-12 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 9.72e-14 ≰ 0.0e+00
    |g(x)|                 = 7.84e-07 ≤ 1.0e-06

 * Work counters
    Seconds run:   29  (vs limit Inf)
    Iterations:    42
    f(x) calls:    93
    ∇f(x) calls:   93


 ooCMF Iter:   43 Total= -22.986708257880 Active= -51.759521470737 G=     6.42e-06
*ooCMF -22.98670826 


In [9]:
#Save molden file only if you just ran the SCF and have the molecule instance
FermiCG.pyscf_write_molden(mol,lo_ao*U, filename="cmf_he08_02.molden");

In [10]:
#rotate the integrals by the cmf calculation
ints = FermiCG.orbital_rotation(ints, U);

In [11]:
max_roots = 100

#Build Cluster Basis (delta n is here)
cluster_bases = FermiCG.compute_cluster_eigenbasis(ints, clusters, verbose=1, max_roots=max_roots, init_fspace=init_fspace, rdm1a=Da, rdm1b=Db);

IDX001:DIM1024:001|002|003|004|005|
 Number of embedded electrons a,b: 7.000000 7.000000 FCIProblem:: #Orbs = 5   #α = 0  #β = 0  Dimension: 1        
   State    1 Energy:   0.00000000  28.77281321
 FCIProblem:: #Orbs = 5   #α = 0  #β = 1  Dimension: 5        
   State    1 Energy:  -2.00393102  26.76888219
   State    2 Energy:   0.09298087  28.86579409
   State    3 Energy:   0.81313385  29.58594706
   State    4 Energy:   0.81313385  29.58594706
   State    5 Energy:   0.82917121  29.60198442
 FCIProblem:: #Orbs = 5   #α = 0  #β = 2  Dimension: 10       
   State    1 Energy:  -1.26690278  27.50591044
   State    2 Energy:  -0.36633091  28.40648230
   State    3 Energy:  -0.36633091  28.40648230
   State    4 Energy:  -0.35178366  28.42102956
   State    5 Energy:   1.61703235  30.38984557
   State    6 Energy:   1.61703235  30.38984557
   State    7 Energy:   1.63144659  30.40425981
   State    8 Energy:   2.47370427  31.24651748
   State    9 Energy:   2.48808879  31.26090200
   

 FCIProblem:: #Orbs = 5   #α = 1  #β = 4  Dimension: 25       
   State    1 Energy:   5.86197242  34.63478564
   State    2 Energy:   5.87419338  34.64700660
   State    3 Energy:   5.87419339  34.64700660
   State    4 Energy:   6.96390404  35.73671725
   State    5 Energy:   7.82584511  36.59865832
   State    6 Energy:   7.84015770  36.61297092
   State    7 Energy:   7.84015771  36.61297092
   State    8 Energy:   8.33057861  37.10339182
   State    9 Energy:   8.73218242  37.50499563
   State   10 Energy:   8.73218242  37.50499563
   State   11 Energy:   8.73255938  37.50537259
   State   12 Energy:   8.73341034  37.50622355
   State   13 Energy:   8.73341034  37.50622355
   State   14 Energy:   8.73766585  37.51047906
   State   15 Energy:   8.86029060  37.63310382
   State   16 Energy:   8.86109452  37.63390773
   State   17 Energy:   8.86109452  37.63390773
   State   18 Energy:   9.12906245  37.90187566
   State   19 Energy:   9.88474363  38.65755685
   State   20 Energy:   9

 FCIProblem:: #Orbs = 5   #α = 2  #β = 3  Dimension: 100      
   State    1 Energy:   5.10667540  33.87948861
   State    2 Energy:   5.10667540  33.87948861
   State    3 Energy:   5.12040164  33.89321485
   State    4 Energy:   5.86197242  34.63478564
   State    5 Energy:   5.87419338  34.64700660
   State    6 Energy:   5.87419339  34.64700660
   State    7 Energy:   6.04342360  34.81623681
   State    8 Energy:   6.05380803  34.82662124
   State    9 Energy:   6.05380803  34.82662124
   State   10 Energy:   6.05766076  34.83047397
   State   11 Energy:   6.05766076  34.83047397
   State   12 Energy:   6.06631870  34.83913192
   State   13 Energy:   6.06631871  34.83913192
   State   14 Energy:   6.07027232  34.84308553
   State   15 Energy:   6.23889690  35.01171011
   State   16 Energy:   6.96390404  35.73671725
   State   17 Energy:   7.16591509  35.93872830
   State   18 Energy:   7.16591509  35.93872831
   State   19 Energy:   7.16646691  35.93928012
   State   20 Energy:   7

 FCIProblem:: #Orbs = 5   #α = 3  #β = 1  Dimension: 50       
   State    1 Energy:   1.85657207  30.62938528
   State    2 Energy:   1.85657207  30.62938528
   State    3 Energy:   1.86930397  30.64211718
   State    4 Energy:   2.90895913  31.68177234
   State    5 Energy:   2.92170123  31.69451444
   State    6 Energy:   2.92170123  31.69451444
   State    7 Energy:   3.68926412  32.46207733
   State    8 Energy:   3.68926412  32.46207733
   State    9 Energy:   3.70354994  32.47636315
   State   10 Energy:   4.27573010  33.04854331
   State   11 Energy:   4.28884057  33.06165378
   State   12 Energy:   4.28884057  33.06165378
   State   13 Energy:   4.60470017  33.37751338
   State   14 Energy:   4.61526078  33.38807400
   State   15 Energy:   4.61526078  33.38807400
   State   16 Energy:   4.61926584  33.39207905
   State   17 Energy:   4.61926584  33.39207905
   State   18 Energy:   4.62780360  33.40061681
   State   19 Energy:   4.62780360  33.40061681
   State   20 Energy:   4

 FCIProblem:: #Orbs = 5   #α = 3  #β = 4  Dimension: 50       
   State    1 Energy:  15.86253490  44.63534811
   State    2 Energy:  16.05139293  44.82420614
   State    3 Energy:  16.05139293  44.82420614
   State    4 Energy:  16.05241294  44.82522616
   State    5 Energy:  16.05301736  44.82583057
   State    6 Energy:  16.05301736  44.82583057
   State    7 Energy:  16.16282729  44.93564050
   State    8 Energy:  16.16403883  44.93685204
   State    9 Energy:  16.16403883  44.93685204
   State   10 Energy:  16.69198324  45.46479645
   State   11 Energy:  16.69198324  45.46479646
   State   12 Energy:  16.70525924  45.47807245
   State   13 Energy:  16.90293759  45.67575080
   State   14 Energy:  16.90704574  45.67985895
   State   15 Energy:  16.90704575  45.67985896
   State   16 Energy:  16.91989470  45.69270791
   State   17 Energy:  16.91989470  45.69270791
   State   18 Energy:  16.94390766  45.71672087
   State   19 Energy:  16.94390767  45.71672088
   State   20 Energy:  16

...

 FCIProblem:: #Orbs = 5   #α = 1  #β = 3  Dimension: 50       
   State    1 Energy:   1.85657206  30.62938527
   State    2 Energy:   1.85657206  30.62938527
   State    3 Energy:   1.86930396  30.64211717
   State    4 Energy:   2.90895911  31.68177233
   State    5 Energy:   2.92170122  31.69451443
   State    6 Energy:   2.92170122  31.69451443
   State    7 Energy:   3.68926410  32.46207731
   State    8 Energy:   3.68926410  32.46207731
   State    9 Energy:   3.70354993  32.47636314
   State   10 Energy:   4.27573008  33.04854330
   State   11 Energy:   4.28884055  33.06165377
   State   12 Energy:   4.28884056  33.06165377
   State   13 Energy:   4.60470016  33.37751337
   State   14 Energy:   4.61526077  33.38807398
   State   15 Energy:   4.61526077  33.38807398
   State   16 Energy:   4.61926582  33.39207904
   State   17 Energy:   4.61926583  33.39207904
   State   18 Energy:   4.62780358  33.40061680
   State   19 Energy:   4.62780359  33.40061680
   State   20 Energy:   4

 FCIProblem:: #Orbs = 5   #α = 2  #β = 3  Dimension: 100      
   State    1 Energy:   5.10667538  33.87948859
   State    2 Energy:   5.10667538  33.87948860
   State    3 Energy:   5.12040163  33.89321484
   State    4 Energy:   5.86197241  34.63478562
   State    5 Energy:   5.87419337  34.64700658
   State    6 Energy:   5.87419338  34.64700659
   State    7 Energy:   6.04342358  34.81623680
   State    8 Energy:   6.05380801  34.82662123
   State    9 Energy:   6.05380801  34.82662123
   State   10 Energy:   6.05766074  34.83047396
   State   11 Energy:   6.05766075  34.83047396
   State   12 Energy:   6.06631869  34.83913190
   State   13 Energy:   6.06631870  34.83913191
   State   14 Energy:   6.07027231  34.84308552
   State   15 Energy:   6.23889688  35.01171010
   State   16 Energy:   6.96390403  35.73671724
   State   17 Energy:   7.16591508  35.93872829
   State   18 Energy:   7.16591508  35.93872829
   State   19 Energy:   7.16646690  35.93928011
   State   20 Energy:   7

 FCIProblem:: #Orbs = 5   #α = 3  #β = 1  Dimension: 50       
   State    1 Energy:   1.85657206  30.62938527
   State    2 Energy:   1.85657206  30.62938527
   State    3 Energy:   1.86930396  30.64211717
   State    4 Energy:   2.90895911  31.68177233
   State    5 Energy:   2.92170122  31.69451443
   State    6 Energy:   2.92170122  31.69451443
   State    7 Energy:   3.68926410  32.46207731
   State    8 Energy:   3.68926410  32.46207731
   State    9 Energy:   3.70354993  32.47636314
   State   10 Energy:   4.27573008  33.04854330
   State   11 Energy:   4.28884055  33.06165377
   State   12 Energy:   4.28884056  33.06165377
   State   13 Energy:   4.60470016  33.37751337
   State   14 Energy:   4.61526077  33.38807398
   State   15 Energy:   4.61526077  33.38807398
   State   16 Energy:   4.61926582  33.39207904
   State   17 Energy:   4.61926583  33.39207904
   State   18 Energy:   4.62780358  33.40061680
   State   19 Energy:   4.62780359  33.40061680
   State   20 Energy:   4

 FCIProblem:: #Orbs = 5   #α = 3  #β = 4  Dimension: 50       
   State    1 Energy:  15.86253488  44.63534809
   State    2 Energy:  16.05139290  44.82420611
   State    3 Energy:  16.05139290  44.82420612
   State    4 Energy:  16.05241292  44.82522613
   State    5 Energy:  16.05301733  44.82583055
   State    6 Energy:  16.05301734  44.82583055
   State    7 Energy:  16.16282727  44.93564048
   State    8 Energy:  16.16403880  44.93685202
   State    9 Energy:  16.16403881  44.93685202
   State   10 Energy:  16.69198322  45.46479643
   State   11 Energy:  16.69198323  45.46479644
   State   12 Energy:  16.70525923  45.47807244
   State   13 Energy:  16.90293756  45.67575078
   State   14 Energy:  16.90704572  45.67985893
   State   15 Energy:  16.90704573  45.67985894
   State   16 Energy:  16.91989469  45.69270790
   State   17 Energy:  16.91989469  45.69270790
   State   18 Energy:  16.94390764  45.71672085
   State   19 Energy:  16.94390765  45.71672086
   State   20 Energy:  16

 FCIProblem:: #Orbs = 5   #α = 4  #β = 4  Dimension: 25       
   State    1 Energy:  22.81452330  51.58733652
   State    2 Energy:  22.81452332  51.58733653
   State    3 Energy:  22.82828220  51.60109541
   State    4 Energy:  22.93633223  51.70914544
   State    5 Energy:  22.94125685  51.71407006
   State    6 Energy:  22.94125686  51.71407007
   State    7 Energy:  22.95535091  51.72816412
   State    8 Energy:  22.95535091  51.72816412
   State    9 Energy:  23.10846889  51.88128211
   State   10 Energy:  23.54764630  52.32045951
   State   11 Energy:  23.56154350  52.33435672
   State   12 Energy:  23.56154352  52.33435673
   State   13 Energy:  23.73505451  52.50786772
   State   14 Energy:  23.74434867  52.51716188
   State   15 Energy:  23.74434868  52.51716190
   State   16 Energy:  24.77331544  53.54612865
   State   17 Energy:  26.44084642  55.21365963
   State   18 Energy:  26.45375318  55.22656639
   State   19 Energy:  26.45375320  55.22656641
   State   20 Energy:  26

In [12]:
#Build Clustered Operator
cluster_ham = FermiCG.extract_ClusteredTerms(ints, clusters);

 Remove duplicates
 Number of terms reduced from 16472 to  6784


In [13]:
#Build Cluster Operators
cluster_ops = FermiCG.compute_cluster_ops(cluster_bases, ints);
FermiCG.add_cmf_operators!(cluster_ops, cluster_bases, ints, Da, Db);

In [14]:
#Need to find reference state 
ref_fock = FermiCG.FockConfig(init_fspace)
nroots = 3
#ci_vector = FermiCG.TPSCIstate(clusters, ref_fock, R=nroots)
ci_vector = FermiCG.TPSCIstate(clusters, FermiCG.FockConfig(init_fspace), R=nroots);
#ci_vector = FermiCG.ClusteredState(clusters, ref_fock, R=nroots);
#Need to find the automated way to define these other excited configs away from ref state, example is to large
#to do by hand
#probably something to do with building p spaces and q spaces
ci_vector[ref_fock][ClusterConfig([1,1,1,1,1,1,1,1])] = [1,0,0]
ci_vector[ref_fock][ClusterConfig([2,1,1,1,1,1,1,1])] = [0,1,0]
ci_vector[ref_fock][ClusterConfig([1,2,1,1,1,1,1,1])] = [0,0,1]

display(ci_vector)

e0, v0 = FermiCG.tpsci_ci(ci_vector, cluster_ops, cluster_ham,
                            thresh_cipsi=1e-3, # Threshold for adding to P-space
                            thresh_foi=1e-7,    # Threshold for keeping terms when defining FOIS
                            thresh_asci=0.001,     # Threshold of P-space configs to search from
                            max_iter=10,
                            matvec=3);


 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =     3  
 ----------                root ------:     =     1  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 1.000               3                   ( 1,1 )( 1,1 )( 1,1 )( 1,1 )( 1,1 )( 1,1 )( 1,1 )( 1,1 )
 --------------------------------------------------
 ci_vector     : (3, 3)
 thresh_cipsi  : 0.001
 thresh_foi    : 1.0e-7
 thresh_asci   : 0.001
 thresh_var    : -1.0
 max_iter      : 10
 conv_thresh   : 0.0001
 nbody         : 4
 incremental   : true
 ci_conv       : 1.0e-5
 ci_max_iter   : 50
 ci_max_ss_vecs: 12
 davidson      : false
 max_mem_ci    : 20.0
 matvec        : 3


     Selected CI Iteration:    1 epsilon:   0.00100000
 Memory needed to hold full CI matrix:   0.00000007 (Gb)

 |== Tensor Product Sta

  Root       Energy           S2
     1 -51.76917718   0.00000002
     2 -50.28813764   1.99999951
     3 -50.25333436   1.99999982
 ==================================================================|
 Length of ASCI vector     1242 →     1099 
 Norm of delta v:
   0.04514625
   1.41328631
   1.41273475
 In open_matvec_thread2
 Setup threaded jobs:                                0.161904 seconds (591.07 k allocations: 58.378 MiB, 20.45% gc time)
 Number of jobs:    90481
 Number of threads: 16
 Compute matrix-vector:                             80.482155 seconds (121.55 M allocations: 6.525 GiB, 5.24% gc time)
 Now collect thread results:                        96.814254 seconds (103.53 M allocations: 7.394 GiB, 74.55% gc time)
 Length of FOIS vector: 30653458
 Compute diagonal:                                  87.734188 seconds (1.99 G allocations: 51.844 GiB, 24.79% gc time)

  Root         E(0)         E(2)
     1 -51.76917718 -52.74009015
     2 -50.28813764 22597.76003905
     3 -

 Length of ASCI vector     1704 →     1088 
 Norm of delta v:
   0.00746715
   0.64291105
   0.61725241
 In open_matvec_thread2
 Setup threaded jobs:                                1.813254 seconds (1.14 M allocations: 99.040 MiB, 85.92% gc time)
 Number of jobs:    153833
 Number of threads: 16
 Compute matrix-vector:                            101.170971 seconds (152.79 M allocations: 7.909 GiB, 4.67% gc time)
 Now collect thread results:                       154.607331 seconds (111.71 M allocations: 8.202 GiB, 82.13% gc time)
 Length of FOIS vector: 40128705
 Compute diagonal:                                 116.955587 seconds (2.61 G allocations: 67.869 GiB, 27.14% gc time)

  Root         E(0)         E(2)
     1 -51.76936896 -51.80702820
     2 -50.33059969 2042.97140921
     3 -50.29087156 2970.90590958
 Length of PT1  vector 40128705 →     1655 
 TPSCI Iter 3   Dim: 1704   E(var):  -51.76936896  -50.33059969  -50.29087156 


     Selected CI Iteration:    4 epsilon:   0.001000

 Setup threaded jobs:                                2.713688 seconds (1.49 M allocations: 148.121 MiB, 86.39% gc time)
 Number of jobs:    210821
 Number of threads: 16
 Compute matrix-vector:                            105.832274 seconds (153.24 M allocations: 8.165 GiB, 2.75% gc time)
 Now collect thread results:                       220.070315 seconds (118.22 M allocations: 8.704 GiB, 83.16% gc time)
 Length of FOIS vector: 45246832
 Compute diagonal:                                 187.719134 seconds (2.94 G allocations: 76.525 GiB, 38.41% gc time)

  Root         E(0)         E(2)
     1 -51.76942685 -51.77869483
     2 -50.33192704 -49.70937658
     3 -50.29273848 13172.91955596
 Length of PT1  vector 45246832 →     1914 
 TPSCI Iter 4   Dim: 1922   E(var):  -51.76942685  -50.33192704  -50.29273848 


     Selected CI Iteration:    5 epsilon:   0.00100000
 Clip values < -1.0e+00           1922 →   1922
 Add pt vector to current space   1922 →   2207
 Memory needed to hold full 

 Compute matrix-vector:                             52.300249 seconds (123.16 M allocations: 5.261 GiB, 24.75% gc time)
 Now collect thread results:                        56.392116 seconds (35.13 M allocations: 2.915 GiB, 85.90% gc time)
 Length of FOIS vector: 48126562
 Compute diagonal:                                 161.159205 seconds (3.13 G allocations: 81.396 GiB, 30.24% gc time)

  Root         E(0)         E(2)
     1 -51.76953863 -51.78226725
     2 -50.33267519 -44.83839837
     3 -50.29444846 -33.71255746
 Length of PT1  vector 48126562 →     1308 
 TPSCI Iter 5   Dim: 2207   E(var):  -51.76953863  -50.33267519  -50.29444846 


     Selected CI Iteration:    6 epsilon:   0.00100000
 Clip values < -1.0e+00           2207 →   2207
 Add pt vector to current space   2207 →   2239
 Memory needed to hold full CI matrix:   0.04010497 (Gb)

 |== Tensor Product State CI =======================================
 Hamiltonian matrix dimension =  2239: 
 Fill old/old Hamiltonian:       

 Length of FOIS vector: 48330754
 Compute diagonal:                                 158.210700 seconds (3.14 G allocations: 81.741 GiB, 29.64% gc time)

  Root         E(0)         E(2)
     1 -51.76957040 -51.77663838
     2 -50.33272344 -50.48737483
     3 -50.29455263 -43.81007356
 Length of PT1  vector 48330754 →      880 
 TPSCI Iter 6   Dim: 2239   E(var):  -51.76957040  -50.33272344  -50.29455263 


     Selected CI Iteration:    7 epsilon:   0.00100000
 Clip values < -1.0e+00           2239 →   2239
 Add pt vector to current space   2239 →   2244
 Memory needed to hold full CI matrix:   0.04028429 (Gb)

 |== Tensor Product State CI =======================================
 Hamiltonian matrix dimension =  2244: 
 Fill old/old Hamiltonian:                           0.151031 seconds (4 allocations: 35.156 KiB)
 Build old/new Hamiltonian matrix with dimension:    0.022667 seconds (215.50 k allocations: 8.306 MiB)
 Build new/new Hamiltonian matrix with dimension:    0.001534 seconds 

     1 -51.76957063 -51.77562187
     2 -50.33273308 -50.50232969
     3 -50.29456574 -50.18829628
 Length of PT1  vector 48384810 →      626 
*TPSCI Iter 7   Dim: 2244   E(var):  -51.76957063  -50.33273308  -50.29456574 


In [15]:
@time e2 = FermiCG.compute_pt2_energy(v0, cluster_ops, cluster_ham, thresh_foi=1e-8)


 |........................do batched PT2............................
 thresh_foi    :1.0e-8
 prescreen     :true
 H0            :Hcmf
 nbody         :4
 Norms of input states:
   1.00000000
   1.00000000
   1.00000000
 Compute <0|H0|0>:                                   0.072877 seconds (4.72 M allocations: 179.851 MiB)
 Compute <0|H|0>:                                    3.317391 seconds (207.75 M allocations: 7.563 GiB, 49.53% gc time)
 Number of jobs:    210821
 Number of threads: 16
   |----------------------------------------------------------------------------------------------------|
   |0%                                                                                              100%|
   |----------------------------------------------------------------------------------------------------|
 Time spent computing E2        583.9 (s)
  Root         E(0)         E(2)
     1 -51.76957063 -51.77161228
     2 -50.33273308 -50.34273993
     3 -50.29456574 -50.30179968
 ..............

3-element Vector{Float64}:
 -0.0020416518263228385
 -0.010006848832230965
 -0.007233938675060941